In [1]:
import requests
import pandas as pd
from pandas.io.html import read_html

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
page='https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods'
wikitable = read_html(page, index_col=0, attrs={"class":"wikitable sortable"})
print("Extracted {num} infoboxes".format(num=len(wikitable)))

Extracted 1 infoboxes


In [4]:
df=wikitable[0]

In [5]:
df.head()

,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map
Name,,,,,,,,,,,,
Toronto CMA Average,NaN,All,5113149,5903.63,866,9.0,40704,10.6,11.4,NaN,NaN,NaN
Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577,12.45,3580,4.6,25750,11.1,5.9,Cantonese (19.3%),19.3% Cantonese,NaN
Alderwood,E,"0211.00, 0212.00",11656,4.94,2360,-4.0,35239,8.8,8.5,Polish (6.2%),06.2% Polish,NaN
Alexandra Park,OCoT,0039.00,4355,0.32,13609,0.0,19687,13.8,28.0,Cantonese (17.9%),17.9% Cantonese,NaN
Allenby,OCoT,0140.00,2513,0.58,4333,-1.0,245592,5.2,3.4,Russian (1.4%),01.4% Russian,NaN


In [6]:
df=df.iloc[:,:1]

In [7]:
df=df.drop(['Toronto CMA Average'],axis=0)

In [8]:
df.replace(to_replace=['OCoT','S','NY','E','Y','EY'],value=['Old City of Toronto','Scarborough','North York','Etobicoke','York','East York'],inplace=True)

In [9]:
df.head()

,FM
Name,
Agincourt,Scarborough
Alderwood,Etobicoke
Alexandra Park,Old City of Toronto
Allenby,Old City of Toronto
Amesbury,North York


In [10]:
df=df.reset_index()

In [11]:
df['FM'].head()

0            Scarborough
1              Etobicoke
2    Old City of Toronto
3    Old City of Toronto
4             North York
Name: FM, dtype: object

In [12]:
latt=[]
long=[]

In [13]:
def extract(name) :   
    page='https://en.wikipedia.org'+name
    #print(page)
    infoboxes = read_html(page, index_col=0, attrs={"class":"infobox"})
    #print("Extracted {num} infoboxes".format(num=len(infoboxes)))
    df=infoboxes[0]
    cname=df.columns[0]
    df.reset_index()
    for x in range(3,7) :
        col=df[cname][x]
        if 'Coordinates' in col :
            st=col
            start=st.find('/ ',25)+3
            end=st.find('W',start)
            coord= st[start:end]
            #print(coord)
            n=coord.find('N')
            #print(n)
            lat=float(coord[:(n-1)])
            lng=-float(coord[(n+1):-1])
            return(lat,lng)

In [14]:
from bs4 import BeautifulSoup
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods").text
soup = BeautifulSoup(website_url,'lxml')

In [15]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [16]:
links=My_table.findAll('a')

In [17]:
lsi = []
for link in links : 
    lsi.append(link.get('href'))

In [18]:
lsi.remove('/wiki/Census_metropolitan_area')

In [19]:
for x in lsi :
    if 'png' in x or 'PNG' in x:
        lsi.remove(x)

In [20]:
df.shape

(174, 2)

In [21]:
for i in range(0,174) :
    try :
        l1,l2= extract(lsi[i])
        latt.append(l1)
        long.append(l2)
    except :
        latt.append('Nan')
        long.append('Nan')

In [22]:
len(latt)

174

In [23]:
df['latitude']=latt

In [24]:
df.head()

,Name,FM,latitude
0,Agincourt,Scarborough,43.7925
1,Alderwood,Etobicoke,43.6075
2,Alexandra Park,Old City of Toronto,43.65
3,Allenby,Old City of Toronto,43.7097
4,Amesbury,North York,43.7006


In [25]:
df['longitude']=long

In [26]:
df.head()

,Name,FM,latitude,longitude
0,Agincourt,Scarborough,43.7925,-79.2839
1,Alderwood,Etobicoke,43.6075,-79.5403
2,Alexandra Park,Old City of Toronto,43.65,-79.4
3,Allenby,Old City of Toronto,43.7097,-79.4151
4,Amesbury,North York,43.7006,-79.4903


In [27]:
for y,x in zip(df['Name'],df['latitude']) :
    if x is 'Nan' :
        print(y)

Armour Heights
Bay Street Corridor
Bayview Village
Bayview Woods – Steeles
Bedford Park
Bloor West Village
Branson
Caribou Park
Centennial
Chaplin Estates
Church and Wellesley
Cricket Club
Deer Park
Discovery District
Distillery District/West Don Lands
Dorset Park
East Danforth
Elia (Jane and Finch)
Fairbank
Financial District
Governor's Bridge/Bennington Heights
Grange Park
Graydon Hall
Guildwood
Harbourfront / CityPlace
High Park North
Highland Creek
Hillcrest
Humber Bay Shores
Humber Heights
Humberlea
Humbermede
Humberwood
Humewood–Cedarvale
Ionview
Kensington Market
Kingsview Village
Lansing
Leslieville
Little Italy
Little Portugal
Long Branch
Malvern
New Toronto
Oakridge
Old Mill/Baby Point
Parkway Forest
Pleasant View
Princess Gardens
Regent Park/Trefann Court
Riverdale
Rouge Hill
Runnymede
Silverthorn
The Beaches
The Danforth
The Elms
The Junction
The Kingsway
Upper Beaches
West Hill
Westmount
Willowdale
York University Heights
Yorkville


In [28]:
for x,y,z in zip(df['Name'],df['latitude'],df['longitude']) :
    if x == 'Armour Heights' :
                 y = 43.7454
                 z = -79.4226

In [29]:
print(df[df['Name'] == 'Armour Heights'])
df.loc[5,'latitude']=43.7454
df.loc[5,'longitude']=-79.4226
print(df[df['Name'] == 'Armour Heights'])

             Name          FM latitude longitude
5  Armour Heights  North York      Nan       Nan
             Name          FM latitude longitude
5  Armour Heights  North York  43.7454  -79.4226


In [30]:
print(df[df['Name'] == 'Bay Street Corridor'])
df.loc[8,'latitude']=43.653908
df.loc[8,'longitude']=-79.384293
print(df[df['Name'] == 'Bay Street Corridor'])

                  Name                   FM latitude longitude
8  Bay Street Corridor  Old City of Toronto      Nan       Nan
                  Name                   FM latitude longitude
8  Bay Street Corridor  Old City of Toronto  43.6539  -79.3843


In [31]:
print(df[df['Name'] == 'Bayview Village'])

              Name          FM latitude longitude
9  Bayview Village  North York      Nan       Nan


In [32]:
df.loc[9,'latitude']=43.7782
df.loc[9,'longitude']=-79.3828
print(df[df['Name'] == 'Bayview Village'])

              Name          FM latitude longitude
9  Bayview Village  North York  43.7782  -79.3828


In [33]:
print(df[df['Name'] == 'Bayview Woods – Steeles'])

                       Name          FM latitude longitude
10  Bayview Woods – Steeles  North York      Nan       Nan


In [34]:
df.loc[10,'latitude']=43.797
df.loc[10,'longitude']=-79.383
print(df[df['Name'] == 'Bayview Woods – Steeles'])

                       Name          FM latitude longitude
10  Bayview Woods – Steeles  North York   43.797   -79.383


In [35]:
print(df[df['Name'] == 'Bedford Park'])

            Name                   FM latitude longitude
11  Bedford Park  Old City of Toronto      Nan       Nan


In [36]:
df.loc[11,'latitude']=43.73
df.loc[11,'longitude']=-79.411
print(df[df['Name'] == 'Bedford Park'])

            Name                   FM latitude longitude
11  Bedford Park  Old City of Toronto    43.73   -79.411


In [37]:
print(df[df['Name'] == 'Bloor West Village'])

                  Name                   FM latitude longitude
14  Bloor West Village  Old City of Toronto      Nan       Nan


In [38]:
df.loc[14,'latitude']=43.650833
df.loc[14,'longitude']=-79.4775
print(df[df['Name'] == 'Bloor West Village'])

                  Name                   FM latitude longitude
14  Bloor West Village  Old City of Toronto  43.6508  -79.4775


In [39]:
print(df[df['Name'] == 'Branson'])

       Name          FM latitude longitude
16  Branson  North York      Nan       Nan


In [40]:
df.loc[16,'latitude']=43.656
df.loc[16,'longitude']=-79.475
print(df[df['Name'] == 'Branson'])

       Name          FM latitude longitude
16  Branson  North York   43.656   -79.475


In [41]:
print(df[df['Name'] == 'Caribou Park'])

            Name          FM latitude longitude
20  Caribou Park  North York      Nan       Nan


In [42]:
df.loc[20,'latitude']=43.7183
df.loc[20,'longitude']=-79.4240
print(df[df['Name'] == 'Caribou Park'])

            Name          FM latitude longitude
20  Caribou Park  North York  43.7183   -79.424


In [43]:
print(df[df['Name'] == 'Centennial'])
df.loc[23,'latitude']=43.7854
df.loc[23,'longitude']=-79.2265
print(df[df['Name'] == 'Centennial'])

          Name         FM latitude longitude
23  Centennial  Etobicoke      Nan       Nan
          Name         FM latitude longitude
23  Centennial  Etobicoke  43.7854  -79.2265


In [44]:
print(df[df['Name'] == 'Chaplin Estates'])
df.loc[24,'latitude']=43.7020
df.loc[24,'longitude']=-79.4015
print(df[df['Name'] == 'Chaplin Estates'])

               Name                   FM latitude longitude
24  Chaplin Estates  Old City of Toronto      Nan       Nan
               Name                   FM latitude longitude
24  Chaplin Estates  Old City of Toronto   43.702  -79.4015


In [45]:
print(df[df['Name'] == 'Church and Wellesley'])
df.loc[26,'latitude']=43.6645
df.loc[26,'longitude']=-79.3818
print(df[df['Name'] == 'Church and Wellesley'])

                    Name                   FM latitude longitude
26  Church and Wellesley  Old City of Toronto      Nan       Nan
                    Name                   FM latitude longitude
26  Church and Wellesley  Old City of Toronto  43.6645  -79.3818


In [46]:
print(df[df['Name'] == 'Cricket Club'])
df.loc[33,'latitude']=43.7385
df.loc[33,'longitude']=-79.4127
print(df[df['Name'] == 'Cricket Club'])

            Name          FM latitude longitude
33  Cricket Club  North York      Nan       Nan
            Name          FM latitude longitude
33  Cricket Club  North York  43.7385  -79.4127


In [47]:
print(df[df['Name'] == 'Deer Park'])
df.loc[36,'latitude']=43.7782
df.loc[36,'longitude']=-79.3828
print(df[df['Name'] == 'Deer Park'])

         Name                   FM latitude longitude
36  Deer Park  Old City of Toronto      Nan       Nan
         Name                   FM latitude longitude
36  Deer Park  Old City of Toronto  43.7782  -79.3828


In [48]:
print(df[df['Name'] == 'Discovery District'])
df.loc[37,'latitude']=43.6603
df.loc[37,'longitude']=-79.3937
print(df[df['Name'] == 'Discovery District'])

                  Name                   FM latitude longitude
37  Discovery District  Old City of Toronto      Nan       Nan
                  Name                   FM latitude longitude
37  Discovery District  Old City of Toronto  43.6603  -79.3937


In [49]:
print(df[df['Name'] == 'Distillery District/West Don Lands'])
df.loc[38,'latitude']=43.6548
df.loc[38,'longitude']=-79.3536
print(df[df['Name'] == 'Distillery District/West Don Lands'])

                                  Name                   FM latitude longitude
38  Distillery District/West Don Lands  Old City of Toronto      Nan       Nan
                                  Name                   FM latitude longitude
38  Distillery District/West Don Lands  Old City of Toronto  43.6548  -79.3536


In [50]:
print(df[df['Name'] == 'Dorset Park'])
df.loc[41,'latitude']=43.7540
df.loc[41,'longitude']=-79.2821
print(df[df['Name'] == 'Dorset Park'])

           Name           FM latitude longitude
41  Dorset Park  Scarborough      Nan       Nan
           Name           FM latitude longitude
41  Dorset Park  Scarborough   43.754  -79.2821


In [51]:
print(df[df['Name'] == 'East Danforth'])
df.loc[46,'latitude']=43.6838
df.loc[46,'longitude']=-79.3212
print(df[df['Name'] == 'East Danforth'])

             Name                   FM latitude longitude
46  East Danforth  Old City of Toronto      Nan       Nan
             Name                   FM latitude longitude
46  East Danforth  Old City of Toronto  43.6838  -79.3212


In [52]:
print(df[df['Name'] == 'Elia (Jane and Finch)'])
df.loc[49,'latitude']=43.7511
df.loc[49,'longitude']=-79.5098
print(df[df['Name'] == 'Elia (Jane and Finch)'])

                     Name          FM latitude longitude
49  Elia (Jane and Finch)  North York      Nan       Nan
                     Name          FM latitude longitude
49  Elia (Jane and Finch)  North York  43.7511  -79.5098


In [53]:
print(df[df['Name'] == 'Fairbank'])
df.loc[51,'latitude']=43.6964
df.loc[51,'longitude']=-79.4563
print(df[df['Name'] == 'Fairbank'])

        Name    FM latitude longitude
51  Fairbank  York      Nan       Nan
        Name    FM latitude longitude
51  Fairbank  York  43.6964  -79.4563


In [54]:
print(df[df['Name'] == 'Financial District'])
df.loc[53,'latitude']=43.6479
df.loc[53,'longitude']=-79.3818
print(df[df['Name'] == 'Financial District'])

                  Name                   FM latitude longitude
53  Financial District  Old City of Toronto      Nan       Nan
                  Name                   FM latitude longitude
53  Financial District  Old City of Toronto  43.6479  -79.3818


In [55]:
print(df[df['Name'] == "Governor's Bridge/Bennington Heights"])
df.loc[59,'latitude']=43.645485
df.loc[59,'longitude']=-79.464752
print(df[df['Name'] == "Governor's Bridge/Bennington Heights"])

                                    Name         FM latitude longitude
59  Governor's Bridge/Bennington Heights  East York      Nan       Nan
                                    Name         FM latitude longitude
59  Governor's Bridge/Bennington Heights  East York  43.6455  -79.4648


In [56]:
print(df[df['Name'] == 'Grange Park'])
df.loc[60,'latitude']=43.6524
df.loc[60,'longitude']=-79.3925
print(df[df['Name'] == 'Grange Park'])

           Name                   FM latitude longitude
60  Grange Park  Old City of Toronto      Nan       Nan
           Name                   FM latitude longitude
60  Grange Park  Old City of Toronto  43.6524  -79.3925


In [57]:
print(df[df['Name'] == 'Graydon Hall'])
df.loc[61,'latitude']=43.7617
df.loc[61,'longitude']=-79.3437
print(df[df['Name'] == 'Graydon Hall'])

            Name          FM latitude longitude
61  Graydon Hall  North York      Nan       Nan
            Name          FM latitude longitude
61  Graydon Hall  North York  43.7617  -79.3437


In [58]:
print(df[df['Name'] == 'Guildwood'])
df.loc[62,'latitude']=43.7527
df.loc[62,'longitude']=-79.1928
print(df[df['Name'] == 'Guildwood'])

         Name           FM latitude longitude
62  Guildwood  Scarborough      Nan       Nan
         Name           FM latitude longitude
62  Guildwood  Scarborough  43.7527  -79.1928


In [59]:
print(df[df['Name'] == 'Harbourfront / CityPlace'])
df.loc[64,'latitude']=43.638
df.loc[64,'longitude']=-79.385
print(df[df['Name'] == 'Harbourfront / CityPlace'])

                        Name                   FM latitude longitude
64  Harbourfront / CityPlace  Old City of Toronto      Nan       Nan
                        Name                   FM latitude longitude
64  Harbourfront / CityPlace  Old City of Toronto   43.638   -79.385


In [60]:
print(df[df['Name'] == 'High Park North'])
df.loc[67,'latitude']=43.6595
df.loc[67,'longitude']=-79.4676
print(df[df['Name'] == 'High Park North'])

               Name                   FM latitude longitude
67  High Park North  Old City of Toronto      Nan       Nan
               Name                   FM latitude longitude
67  High Park North  Old City of Toronto  43.6595  -79.4676


In [61]:
print(df[df['Name'] == 'Highland Creek'])
df.loc[68,'latitude']=43.7866
df.loc[68,'longitude']=-79.1883
print(df[df['Name'] == 'Highland Creek'])

              Name           FM latitude longitude
68  Highland Creek  Scarborough      Nan       Nan
              Name           FM latitude longitude
68  Highland Creek  Scarborough  43.7866  -79.1883


In [62]:
print(df[df['Name'] == 'Hillcrest'])
df.loc[69,'latitude']=43.8049
df.loc[69,'longitude']=-79.3547
print(df[df['Name'] == 'Hillcrest'])

         Name          FM latitude longitude
69  Hillcrest  North York      Nan       Nan
         Name          FM latitude longitude
69  Hillcrest  North York  43.8049  -79.3547


In [63]:
print(df[df['Name'] == 'Humber Bay Shores'])
df.loc[71,'latitude']=43.6260
df.loc[71,'longitude']=-79.4774
print(df[df['Name'] == 'Humber Bay Shores'])

                 Name         FM latitude longitude
71  Humber Bay Shores  Etobicoke      Nan       Nan
                 Name         FM latitude longitude
71  Humber Bay Shores  Etobicoke   43.626  -79.4774


In [64]:
print(df[df['Name'] == 'Humber Heights'])
df.loc[72,'latitude']=43.6964
df.loc[72,'longitude']=-79.5270
print(df[df['Name'] == 'Humber Heights'])

              Name         FM latitude longitude
72  Humber Heights  Etobicoke      Nan       Nan
              Name         FM latitude longitude
72  Humber Heights  Etobicoke  43.6964   -79.527


In [65]:
print(df[df['Name'] == 'Humberlea'])
df.loc[75,'latitude']=43.7213
df.loc[75,'longitude']=-79.5332
print(df[df['Name'] == 'Humberlea'])

         Name          FM latitude longitude
75  Humberlea  North York      Nan       Nan
         Name          FM latitude longitude
75  Humberlea  North York  43.7213  -79.5332


In [66]:
print(df[df['Name'] == 'Humbermede'])
df.loc[76,'latitude']=43.7421
df.loc[76,'longitude']=-79.5407
print(df[df['Name'] == 'Humbermede'])

          Name          FM latitude longitude
76  Humbermede  North York      Nan       Nan
          Name          FM latitude longitude
76  Humbermede  North York  43.7421  -79.5407


In [67]:
print(df[df['Name'] == 'Humberwood'])
df.loc[77,'latitude']=43.724964
df.loc[77,'longitude']=-79.581528
print(df[df['Name'] == 'Humberwood'])

          Name         FM latitude longitude
77  Humberwood  Etobicoke      Nan       Nan
          Name         FM latitude longitude
77  Humberwood  Etobicoke   43.725  -79.5815


In [68]:
print(df[df['Name'] == 'Humewood–Cedarvale'])
df.loc[78,'latitude']=43.6938
df.loc[78,'longitude']=-79.4282
print(df[df['Name'] == 'Humewood–Cedarvale'])

                  Name    FM latitude longitude
78  Humewood–Cedarvale  York      Nan       Nan
                  Name    FM latitude longitude
78  Humewood–Cedarvale  York  43.6938  -79.4282


In [69]:
print(df[df['Name'] == 'Ionview'])
df.loc[79,'latitude']=43.7308
df.loc[79,'longitude']=-79.2739
print(df[df['Name'] == 'Ionview'])

       Name           FM latitude longitude
79  Ionview  Scarborough      Nan       Nan
       Name           FM latitude longitude
79  Ionview  Scarborough  43.7308  -79.2739


In [70]:
print(df[df['Name'] == 'Kensington Market'])
df.loc[82,'latitude']=43.6545
df.loc[82,'longitude']=-79.4015
print(df[df['Name'] == 'Kensington Market'])

                 Name                   FM latitude longitude
82  Kensington Market  Old City of Toronto      Nan       Nan
                 Name                   FM latitude longitude
82  Kensington Market  Old City of Toronto  43.6545  -79.4015


In [71]:
print(df[df['Name'] == 'Kingsview Village'])
df.loc[83,'latitude']=43.7005
df.loc[83,'longitude']=-79.5519
print(df[df['Name'] == 'Kingsview Village'])

                 Name         FM latitude longitude
83  Kingsview Village  Etobicoke      Nan       Nan
                 Name         FM latitude longitude
83  Kingsview Village  Etobicoke  43.7005  -79.5519


In [72]:
print(df[df['Name'] == 'Lansing'])
df.loc[86,'latitude']=43.7596
df.loc[86,'longitude']=-79.4226
print(df[df['Name'] == 'Lansing'])

       Name          FM latitude longitude
86  Lansing  North York      Nan       Nan
       Name          FM latitude longitude
86  Lansing  North York  43.7596  -79.4226


In [73]:
print(df[df['Name'] == 'Leslieville'])
df.loc[92,'latitude']=43.6658
df.loc[92,'longitude']=-79.3325
print(df[df['Name'] == 'Leslieville'])

           Name                   FM latitude longitude
92  Leslieville  Old City of Toronto      Nan       Nan
           Name                   FM latitude longitude
92  Leslieville  Old City of Toronto  43.6658  -79.3325


In [74]:
print(df[df['Name'] == 'Little Italy'])
df.loc[93,'latitude']=43.6548
df.loc[93,'longitude']=-79.4169
print(df[df['Name'] == 'Little Italy'])

            Name                   FM latitude longitude
93  Little Italy  Old City of Toronto      Nan       Nan
            Name                   FM latitude longitude
93  Little Italy  Old City of Toronto  43.6548  -79.4169


In [75]:
print(df[df['Name'] == 'Little Portugal'])
df.loc[94,'latitude']=43.6463
df.loc[94,'longitude']=-79.4282
print(df[df['Name'] == 'Little Portugal'])

               Name                   FM latitude longitude
94  Little Portugal  Old City of Toronto      Nan       Nan
               Name                   FM latitude longitude
94  Little Portugal  Old City of Toronto  43.6463  -79.4282


In [76]:
print(df[df['Name'] == 'Long Branch'])
df.loc[95,'latitude']=43.5934
df.loc[95,'longitude']=-79.5382
print(df[df['Name'] == 'Long Branch'])

           Name         FM latitude longitude
95  Long Branch  Etobicoke      Nan       Nan
           Name         FM latitude longitude
95  Long Branch  Etobicoke  43.5934  -79.5382


In [77]:
print(df[df['Name'] == 'Malvern'])
df.loc[97,'latitude']=43.8092
df.loc[97,'longitude']=-79.2217
print(df[df['Name'] == 'Malvern'])

       Name           FM latitude longitude
97  Malvern  Scarborough      Nan       Nan
       Name           FM latitude longitude
97  Malvern  Scarborough  43.8092  -79.2217


In [78]:
print(df[df['Name'] == 'New Toronto'])
df.loc[106,'latitude']=43.6008
df.loc[106,'longitude']=-79.5053
print(df[df['Name'] == 'New Toronto'])

            Name         FM latitude longitude
106  New Toronto  Etobicoke      Nan       Nan
            Name         FM latitude longitude
106  New Toronto  Etobicoke  43.6008  -79.5053


In [79]:
print(df[df['Name'] == 'Oakridge'])
df.loc[110,'latitude']=43.6966
df.loc[110,'longitude']=-79.2829
print(df[df['Name'] == 'Oakridge'])

         Name           FM latitude longitude
110  Oakridge  Scarborough      Nan       Nan
         Name           FM latitude longitude
110  Oakridge  Scarborough  43.6966  -79.2829


In [80]:
print(df[df['Name'] == 'Old Mill/Baby Point'])
df.loc[113,'latitude']=43.6521 
df.loc[113,'longitude']=-79.4893
print(df[df['Name'] == 'Old Mill/Baby Point'])

                    Name    FM latitude longitude
113  Old Mill/Baby Point  York      Nan       Nan
                    Name    FM latitude longitude
113  Old Mill/Baby Point  York  43.6521  -79.4893


In [81]:
print(df[df['Name'] == 'Parkway Forest'])
df.loc[115,'latitude']=43.7722
df.loc[115,'longitude']=-79.3423
print(df[df['Name'] == 'Parkway Forest'])

               Name          FM latitude longitude
115  Parkway Forest  North York      Nan       Nan
               Name          FM latitude longitude
115  Parkway Forest  North York  43.7722  -79.3423


In [82]:
print(df[df['Name'] == 'Pleasant View'])
df.loc[119,'latitude']=43.7848
df.loc[119,'longitude']=-79.3381
print(df[df['Name'] == 'Pleasant View'])

              Name          FM latitude longitude
119  Pleasant View  North York      Nan       Nan
              Name          FM latitude longitude
119  Pleasant View  North York  43.7848  -79.3381


In [83]:
print(df[df['Name'] == 'Princess Gardens'])
df.loc[122,'latitude']=43.6689
df.loc[122,'longitude']=-79.5435
print(df[df['Name'] == 'Princess Gardens'])

                 Name         FM latitude longitude
122  Princess Gardens  Etobicoke      Nan       Nan
                 Name         FM latitude longitude
122  Princess Gardens  Etobicoke  43.6689  -79.5435


In [84]:
print(df[df['Name'] == 'Caribou Park'])
#df.loc[20,'latitude']=43.7183
#df.loc[20,'longitude']=-79.4240
#print(df[df['Name'] == 'Caribou Park'])

            Name          FM latitude longitude
20  Caribou Park  North York  43.7183   -79.424


In [85]:
for y,x in zip(df['Name'],df['latitude']) :
    if x is 'Nan' :
        print(y)

Regent Park/Trefann Court
Riverdale
Rouge Hill
Runnymede
Silverthorn
The Beaches
The Danforth
The Elms
The Junction
The Kingsway
Upper Beaches
West Hill
Westmount
Willowdale
York University Heights
Yorkville


In [86]:
print(df[df['Name'] == 'Regent Park/Trefann Court'])
df.loc[124,'latitude']=43.6575
df.loc[124,'longitude']=-79.3603
print(df[df['Name'] == 'Regent Park/Trefann Court'])

                          Name                   FM latitude longitude
124  Regent Park/Trefann Court  Old City of Toronto      Nan       Nan
                          Name                   FM latitude longitude
124  Regent Park/Trefann Court  Old City of Toronto  43.6575  -79.3603


In [87]:
print(df[df['Name'] == 'Riverdale'])
df.loc[126,'latitude']=43.6697
df.loc[126,'longitude']=-79.3532
print(df[df['Name'] == 'Riverdale'])

          Name                   FM latitude longitude
126  Riverdale  Old City of Toronto      Nan       Nan
          Name                   FM latitude longitude
126  Riverdale  Old City of Toronto  43.6697  -79.3532


In [88]:
print(df[df['Name'] == 'Rouge Hill'])
df.loc[131,'latitude']=43.7947
df.loc[131,'longitude']=-79.1345
print(df[df['Name'] == 'Rouge Hill'])

           Name           FM latitude longitude
131  Rouge Hill  Scarborough      Nan       Nan
           Name           FM latitude longitude
131  Rouge Hill  Scarborough  43.7947  -79.1345


In [89]:
print(df[df['Name'] == 'Runnymede'])
df.loc[132,'latitude']=43.6617
df.loc[132,'longitude']=-79.4849
print(df[df['Name'] == 'Runnymede'])

          Name    FM latitude longitude
132  Runnymede  York      Nan       Nan
          Name    FM latitude longitude
132  Runnymede  York  43.6617  -79.4849


In [90]:
print(df[df['Name'] == 'Silverthorn'])
df.loc[137,'latitude']=43.6880
df.loc[137,'longitude']=-79.4676
print(df[df['Name'] == 'Silverthorn'])

            Name    FM latitude longitude
137  Silverthorn  York      Nan       Nan
            Name    FM latitude longitude
137  Silverthorn  York   43.688  -79.4676


In [91]:
print(df[df['Name'] == 'The Beaches'])
df.loc[147,'latitude']=43.6673
df.loc[147,'longitude']=-79.2967
print(df[df['Name'] == 'The Beaches'])

            Name                   FM latitude longitude
147  The Beaches  Old City of Toronto      Nan       Nan
            Name                   FM latitude longitude
147  The Beaches  Old City of Toronto  43.6673  -79.2967


In [92]:
print(df[df['Name'] == 'The Danforth'])
df.loc[148,'latitude']=43.6838
df.loc[148,'longitude']=-79.3212
print(df[df['Name'] == 'The Danforth'])

             Name                   FM latitude longitude
148  The Danforth  Old City of Toronto      Nan       Nan
             Name                   FM latitude longitude
148  The Danforth  Old City of Toronto  43.6838  -79.3212


In [93]:
print(df[df['Name'] == 'The Elms'])
df.loc[149,'latitude']=43.7237
df.loc[149,'longitude']=-79.5463
print(df[df['Name'] == 'The Elms'])

         Name         FM latitude longitude
149  The Elms  Etobicoke      Nan       Nan
         Name         FM latitude longitude
149  The Elms  Etobicoke  43.7237  -79.5463


In [94]:
print(df[df['Name'] == 'The Junction'])
df.loc[150,'latitude']=43.6666
df.loc[150,'longitude']=-79.4686
print(df[df['Name'] == 'The Junction'])

             Name                   FM latitude longitude
150  The Junction  Old City of Toronto      Nan       Nan
             Name                   FM latitude longitude
150  The Junction  Old City of Toronto  43.6666  -79.4686


In [95]:
print(df[df['Name'] == 'The Kingsway'])
df.loc[151,'latitude']=43.6527
df.loc[151,'longitude']=-79.5072
print(df[df['Name'] == 'The Kingsway'])

             Name         FM latitude longitude
151  The Kingsway  Etobicoke      Nan       Nan
             Name         FM latitude longitude
151  The Kingsway  Etobicoke  43.6527  -79.5072


In [96]:
print(df[df['Name'] == 'Upper Beaches'])
df.loc[157,'latitude']=43.6780
df.loc[157,'longitude']=-79.3099
print(df[df['Name'] == 'Upper Beaches'])

              Name                   FM latitude longitude
157  Upper Beaches  Old City of Toronto      Nan       Nan
              Name                   FM latitude longitude
157  Upper Beaches  Old City of Toronto   43.678  -79.3099


In [97]:
print(df[df['Name'] == 'West Hill'])
df.loc[161,'latitude']=43.7692
df.loc[161,'longitude']=-79.1828
print(df[df['Name'] == 'West Hill'])

          Name           FM latitude longitude
161  West Hill  Scarborough      Nan       Nan
          Name           FM latitude longitude
161  West Hill  Scarborough  43.7692  -79.1828


In [98]:
print(df[df['Name'] == 'Westmount'])
df.loc[164,'latitude']=43.6871
df.loc[164,'longitude']=-79.5224
print(df[df['Name'] == 'Westmount'])

          Name         FM latitude longitude
164  Westmount  Etobicoke      Nan       Nan
          Name         FM latitude longitude
164  Westmount  Etobicoke  43.6871  -79.5224


In [99]:
print(df[df['Name'] == 'Willowdale'])
df.loc[167,'latitude']=43.7712
df.loc[167,'longitude']=-79.4197
print(df[df['Name'] == 'Willowdale'])

           Name          FM latitude longitude
167  Willowdale  North York      Nan       Nan
           Name          FM latitude longitude
167  Willowdale  North York  43.7712  -79.4197


In [100]:
print(df[df['Name'] == 'York University Heights'])
df.loc[172,'latitude']=43.7680
df.loc[172,'longitude']=-79.4873
print(df[df['Name'] == 'York University Heights'])

                        Name          FM latitude longitude
172  York University Heights  North York      Nan       Nan
                        Name          FM latitude longitude
172  York University Heights  North York   43.768  -79.4873


In [101]:
print(df[df['Name'] == 'Yorkville'])
df.loc[173,'latitude']=43.761539
df.loc[173,'longitude']=-79.411079
print(df[df['Name'] == 'Yorkville'])

          Name                   FM latitude longitude
173  Yorkville  Old City of Toronto      Nan       Nan
          Name                   FM latitude longitude
173  Yorkville  Old City of Toronto  43.7615  -79.4111


In [102]:
df.head()

,Name,FM,latitude,longitude
0,Agincourt,Scarborough,43.7925,-79.2839
1,Alderwood,Etobicoke,43.6075,-79.5403
2,Alexandra Park,Old City of Toronto,43.65,-79.4
3,Allenby,Old City of Toronto,43.7097,-79.4151
4,Amesbury,North York,43.7006,-79.4903


In [103]:
df=df.rename(index=str, columns={"Name": "Neighborhood", "FM": "Borough","latitude":"Latitude","longitude":"Longitude"})

In [104]:
df.head()

,Neighborhood,Borough,Latitude,Longitude
0,Agincourt,Scarborough,43.7925,-79.2839
1,Alderwood,Etobicoke,43.6075,-79.5403
2,Alexandra Park,Old City of Toronto,43.65,-79.4
3,Allenby,Old City of Toronto,43.7097,-79.4151
4,Amesbury,North York,43.7006,-79.4903


In [105]:
neighborhoods=df

In [106]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 6 boroughs and 174 neighborhoods.


In [107]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_0         4.0 MB  conda-forge
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_0
  orderedset         conda-forge/linux-64::orderedset-2.0-py36_0
  ratelim            con

In [108]:
import geocoder

In [109]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [110]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [111]:
CLIENT_ID = '1QKLV2GDIKOCAD1HFKRQDHYNSIVNQYA02RZEFQVJXT0QL5SK' # your Foursquare ID
CLIENT_SECRET = 'O53UIHGDY1ER23NUNWPYLRGVFZ1KJFZUJB4VOO5PHOJSXEXG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1QKLV2GDIKOCAD1HFKRQDHYNSIVNQYA02RZEFQVJXT0QL5SK
CLIENT_SECRET:O53UIHGDY1ER23NUNWPYLRGVFZ1KJFZUJB4VOO5PHOJSXEXG


In [112]:
neighborhoods.loc['0','Neighborhood']

'Agincourt'

In [113]:
neighborhood_latitude = neighborhoods.loc['0', 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc['0', 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc['0', 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Agincourt are 43.7925, -79.28389.


In [114]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude
    )
url
#https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD


'https://api.foursquare.com/v2/venues/search?&client_id=1QKLV2GDIKOCAD1HFKRQDHYNSIVNQYA02RZEFQVJXT0QL5SK&client_secret=O53UIHGDY1ER23NUNWPYLRGVFZ1KJFZUJB4VOO5PHOJSXEXG&v=20180605&ll=43.7925,-79.28389'

In [115]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7e068d4434b957862201d3'},
 'response': {'venues': [{'id': '4baa8ebcf964a5206b743ae3',
    'name': 'Agincourt Collegiate Institute',
    'location': {'address': '2621 Midland Ave',
     'lat': 43.789453273778676,
     'lng': -79.27923523949332,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.789453273778676,
       'lng': -79.27923523949332}],
     'distance': 504,
     'postalCode': 'M1S 1R6',
     'cc': 'CA',
     'city': 'Toronto Division',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['2621 Midland Ave',
      'Toronto Division ON M1S 1R6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d13d941735',
      'name': 'High School',
      'pluralName': 'High Schools',
      'shortName': 'High School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1551763086',
    'hasPerk': False},
   {'id'

In [116]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [117]:
results['response']

{'venues': [{'id': '4baa8ebcf964a5206b743ae3',
   'name': 'Agincourt Collegiate Institute',
   'location': {'address': '2621 Midland Ave',
    'lat': 43.789453273778676,
    'lng': -79.27923523949332,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.789453273778676,
      'lng': -79.27923523949332}],
    'distance': 504,
    'postalCode': 'M1S 1R6',
    'cc': 'CA',
    'city': 'Toronto Division',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['2621 Midland Ave',
     'Toronto Division ON M1S 1R6',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d13d941735',
     'name': 'High School',
     'pluralName': 'High Schools',
     'shortName': 'High School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1551763086',
   'hasPerk': False},
  {'id': '4c4212bb3735be9aba7019a4',
   'name': 'Shell',
   'location': {'address': '2801 Midland Ave',
    'la

In [118]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,categories
0,Agincourt Collegiate Institute,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",43.789453,-79.279235,High School
1,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",43.797544,-79.283345,Gas Station
2,Alexander Family Rez!,[],43.792645,-79.283988,None
3,Donalda Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.790280,-79.283630,Park
4,Agincourt Recreation Centre,"[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",43.791396,-79.272075,Pool


In [119]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Agincourt
Alderwood
Alexandra Park
Allenby
Amesbury
Armour Heights
Banbury
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods – Steeles
Bedford Park
Bendale
Birch Cliff
Bloor West Village
Bracondale Hill
Branson
Bridle Path
Brockton
Cabbagetown
Caribou Park
Carleton Village
Casa Loma
Centennial
Chaplin Estates
Christie Pits
Church and Wellesley
Clairville
Clairlea
Cliffcrest
Cliffside
Corktown
Crescent Town
Cricket Club
Davenport
Davisville
Deer Park
Discovery District
Distillery District/West Don Lands
Don Mills
Don Valley Village
Dorset Park
Dovercourt Park
Downsview
Dufferin Grove
Earlscourt
East Danforth
Eatonville
Eglinton East
Elia (Jane and Finch)
Eringate
Fairbank
Fashion District
Financial District
Flemingdon Park
Forest Hill
Fort York/Liberty Village
Garden District
Glen Park
Governor's Bridge/Bennington Heights
Grange Park
Graydon Hall
Guildwood
Harbord Village
Harbourfront / CityPlace
Harwood
Henry Farm
High Park North
Highland Creek
Hillcrest
Hoggs Hollow
Hum

In [122]:
print(toronto_venues.shape)
toronto_venues.head()

(3865, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.7925,-79.28389,Donalda Park,43.790280,-79.283630,Park
1,Agincourt,43.7925,-79.28389,Havendale Park,43.796010,-79.284398,Park
2,Agincourt,43.7925,-79.28389,LT turf,43.788811,-79.281426,BBQ Joint
3,Alderwood,43.6075,-79.54028,Douglas Park,43.609872,-79.537724,Park
4,Alderwood,43.6075,-79.54028,Alderwood Memorial Park,43.605757,-79.544387,Playground


In [123]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
Alderwood,4,4,4,4,4,4
Alexandra Park,100,100,100,100,100,100
Allenby,2,2,2,2,2,2
Amesbury,2,2,2,2,2,2
Armour Heights,1,1,1,1,1,1
Banbury,37,37,37,37,37,37
Bathurst Manor,4,4,4,4,4,4
Bay Street Corridor,100,100,100,100,100,100


In [124]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 308 uniques categories.


In [145]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()
# add neighborhood column back to dataframe


,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant

In [146]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()


,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant

In [147]:
list1.index('Neighborhood')

199

In [148]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[199]] + list(toronto_onehot.columns[:199])+list(toronto_onehot.columns[200:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant

In [149]:
toronto_onehot.shape

(3865, 308)

In [150]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant

In [151]:
toronto_grouped.shape

(173, 308)

In [152]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0                  Park  0.67
1             BBQ Joint  0.33
2     Accessories Store  0.00
3     Other Repair Shop  0.00
4  Other Great Outdoors  0.00


----Alderwood----
                     venue  freq
0                     Park  0.25
1       Athletics & Sports  0.25
2               Playground  0.25
3                   Market  0.25
4  North Indian Restaurant  0.00


----Alexandra Park----
                           venue  freq
0                            Bar  0.06
1                           Café  0.05
2  Vegetarian / Vegan Restaurant  0.05
3                    Coffee Shop  0.04
4                     Restaurant  0.03


----Allenby----
                     venue  freq
0  Health & Beauty Service   0.5
1                   Garden   0.5
2        Accessories Store   0.0
3        Other Repair Shop   0.0
4     Other Great Outdoors   0.0


----Amesbury----
                  venue  freq
0         Grocery Store   0.5
1           Supermarket   0.5


In [153]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [154]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Park,BBQ Joint,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
1,Alderwood,Market,Park,Athletics & Sports,Playground,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
2,Alexandra Park,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,French Restaurant,Dessert Shop,Record Shop,Cocktail Bar,Yoga Studio
3,Allenby,Garden,Health & Beauty Service,Zoo,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
4,Amesbury,Grocery Store,Supermarket,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit


In [155]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 3, 3, 3, 3, 2, 3, 4], dtype=int32)

In [168]:
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop(['Cluster Labels'],axis=1)

In [169]:
# add clustering labels

neighborhoods_venues_sorted.insert(loc=0, column='Cluster Labels', value=kmeans.labels_)
neighborhoods_venues_sorted.head()


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agincourt,Park,BBQ Joint,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
1,2,Alderwood,Market,Park,Athletics & Sports,Playground,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
2,3,Alexandra Park,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,French Restaurant,Dessert Shop,Record Shop,Cocktail Bar,Yoga Studio
3,3,Allenby,Garden,Health & Beauty Service,Zoo,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
4,3,Amesbury,Grocery Store,Supermarket,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit


In [165]:
for x in range(0,173) :
    neighborhoods_venues_sorted.loc[x,'Cluster Labels']=int(neighborhoods_venues_sorted.loc[x,'Cluster Labels'])

In [170]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agincourt,Park,BBQ Joint,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
1,2,Alderwood,Market,Park,Athletics & Sports,Playground,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
2,3,Alexandra Park,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,French Restaurant,Dessert Shop,Record Shop,Cocktail Bar,Yoga Studio
3,3,Allenby,Garden,Health & Beauty Service,Zoo,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
4,3,Amesbury,Grocery Store,Supermarket,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit


In [162]:
toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Scarborough,43.7925,-79.2839,2.0,Park,BBQ Joint,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
1,Alderwood,Etobicoke,43.6075,-79.5403,2.0,Market,Park,Athletics & Sports,Playground,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
2,Alexandra Park,Old City of Toronto,43.65,-79.4,3.0,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,French Restaurant,Dessert Shop,Record Shop,Cocktail Bar,Yoga Studio
3,Allenby,Old City of Toronto,43.7097,-79.4151,3.0,Garden,Health & Beauty Service,Zoo,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
4,Amesbury,North York,43.7006,-79.4903,3.0,Grocery Store,Supermarket,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit


In [171]:
toronto_merged.shape

(174, 15)

In [173]:
toronto_merged=toronto_merged.fillna(0)

In [180]:
list1=[]
for x in range(0,174) :
    i=str(x)
    list1.append(int(toronto_merged.loc[i,'Cluster Labels']))
        

In [184]:
toronto_merged=toronto_merged.drop(['Cluster Labels'],axis=1)

In [185]:
toronto_merged['Cluster Labels']=list1

In [186]:
toronto_merged.head() 

,Neighborhood,Borough,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Agincourt,Scarborough,43.792500,-79.283890,Park,BBQ Joint,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,2
1,Alderwood,Etobicoke,43.607500,-79.540280,Market,Park,Athletics & Sports,Playground,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,2
2,Alexandra Park,Old City of Toronto,43.650000,-79.400000,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,French Restaurant,Dessert Shop,Record Shop,Cocktail Bar,Yoga Studio,3
3,Allenby,Old City of Toronto,43.709699,-79.415107,Garden,Health & Beauty Service,Zoo,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,3
4,Amesbury,North York,43.700560,-79.490280,Grocery Store,Supermarket,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,3


In [206]:
x='0'
toronto_merged.loc[x,'Cluster Labels']=int(toronto_merged.loc[x,'Cluster Labels'])

3

In [187]:
toronto_merged.shape

(174, 15)

In [188]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [189]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
17,North York,Zoo,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,0
169,Scarborough,0,0,0,0,0,0,0,0,0,0


In [190]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
11,Old City of Toronto,Pizza Place,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,1
12,Scarborough,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Zoo,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,1
29,Scarborough,Discount Store,Wings Joint,Park,Burger Joint,Pizza Place,Pharmacy,Exhibit,Dumpling Restaurant,Eastern European Restaurant,1
48,Scarborough,Home Service,Bar,Hotel,Flea Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,1
91,North York,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Beer Store,Nail Salon,Pharmacy,Bakery,Farmers Market,Field,1
97,Scarborough,Pizza Place,Pharmacy,Bubble Tea Shop,Gym / Fitness Center,Skating Rink,Sandwich Place,Park,Grocery Store,Donut Shop,1
105,York,Discount Store,Pizza Place,Fast Food Restaurant,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,1
110,Scarborough,Chinese Restaurant,Filipino Restaurant,Pizza Place,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,1
117,North York,Breakfast Spot,Pharmacy,Bank,Gastropub,Athletics & Sports,Intersection,Pet Store,Rock Climbing Spot,Fast Food Restaurant,1
129,Old City of Toronto,Playground,Pizza Place,Zoo,Exhibit,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,1


In [191]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Scarborough,BBQ Joint,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,2
1,Etobicoke,Park,Athletics & Sports,Playground,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,2
7,North York,Park,Baseball Field,Playground,Zoo,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,2
10,North York,Park,Flea Market,Exhibit,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,2
13,Scarborough,Park,Thai Restaurant,Diner,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,2
15,Old City of Toronto,Café,Event Space,Farmers Market,Zoo,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Exhibit,2
32,East York,Park,Metro Station,Golf Course,Exhibit,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,2
67,Old City of Toronto,Convenience Store,Italian Restaurant,Café,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,2
72,Etobicoke,Coffee Shop,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,2
77,Etobicoke,Skating Rink,Disc Golf,Zoo,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,2


In [192]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Old City of Toronto,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,French Restaurant,Dessert Shop,Record Shop,Cocktail Bar,Yoga Studio,3
3,Old City of Toronto,Health & Beauty Service,Zoo,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,3
4,North York,Supermarket,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,3
5,North York,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farm,3
6,North York,Shoe Store,Clothing Store,American Restaurant,Restaurant,Bubble Tea Shop,Chocolate Shop,Middle Eastern Restaurant,Furniture / Home Store,Jewelry Store,3
8,Old City of Toronto,Coffee Shop,Steakhouse,Tea Room,Breakfast Spot,Plaza,Cosmetics Shop,American Restaurant,Burger Joint,Bubble Tea Shop,3
14,Old City of Toronto,Coffee Shop,Bakery,Bar,Pizza Place,Sushi Restaurant,Falafel Restaurant,Health & Beauty Service,Spa,Frozen Yogurt Shop,3
16,North York,Greek Restaurant,Liquor Store,Pizza Place,Zoo,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,3
18,Old City of Toronto,Café,Bar,Vietnamese Restaurant,Grocery Store,Bakery,Restaurant,Boutique,Thai Restaurant,Gastropub,3
19,Old City of Toronto,Park,Pharmacy,Beer Store,Japanese Restaurant,Baseball Field,Thai Restaurant,Jewelry Store,Taiwanese Restaurant,Sushi Restaurant,3


In [193]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
9,North York,Zoo,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,4
36,Old City of Toronto,Zoo,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,4
171,North York,Trail,Zoo,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,4
